In [ ]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.message import EmailMessage
from email.utils import make_msgid
import mimetypes

import psycopg2
from psycopg2 import sql

db_params = {
    "host": "localhost",
    # "host": "10.0.0.51",
    "database": "landy",
    "user": "kristoferpearson",
    "password": ""
    # "user": "postgres",
    # "password": "postgres"
}

# file_path = '/Volumes/pdf_files/garf/Kris/Projects/NjCpaExpressWrongRenSolAppFix_20241211/CPA_Express_1022.txt'
file_path = '/Users/kristoferpearson/Desktop/CPA_Express_1022.txt'

# Send the message via our own SMTP server, but don't include the
# envelope header.
smtp_host = 'localhost'

signature_path = ''
applications_path = ''

with open(file_path, 'r') as f:
    for line in f:
        values = line.split("!")
        if len(values) > 2 and values[1] != 'contact' and values[1] != 'Accountants Renewal  0':
            insured_name = values[1].strip()
            insured_email = values[35].strip()
            expiration_date = values[37].strip()
            producer_code = values[43].strip()
            risk_code = values[45].strip()
            policy_number = values[47].strip()
            policy_number_digits = policy_number[3:-3]
            if producer_code != "HHL01-A":
                conn = psycopg2.connect(**db_params)

                # Create a cursor object
                cur = conn.cursor()

                # Execute a query
                query = sql.SQL(f"SELECT \"email[1]\" FROM cpa_coprod WHERE cpcode = '{producer_code}';")
                cur.execute(query)
                producer_email = cur.fetchone()

                msg = EmailMessage()

                # generic email headers
                msg['Subject'] = 'Hello there'
                msg['From'] = 'landy_insurance@landy.com'
                msg['To'] = 'kristofer@landy.com'
                # msg['To'] = producer_email

                # set the plain text body
                msg.set_content(f'''Dear {insured_name}:
                                
Regarding:  {insured_name}

            Accountants Professional Liability Renewal Application

            Policy Number: {policy_number}

            Policy Expiration Date: {expiration_date}

 

Attached please find an amended Accountants Professional Liability Express renewal application for your client's completion to renew their Accountants Professional Liability Insurance policy. In error an outdated version of the application was previously sent to you.

 

Please have your client use the attached renewal application to renew their Accountants Professional Liability policy, as long as they can continue to answer “true” to all the eligibility questions on the application. If for any reason they cannot continue to answer “true to all the eligibility questions on the application, please contact our office for assistance at 800-336-5422.

 

We apologize for this error and any inconvenience this may have caused you.''')
                
                # now create a Content-ID for the image
                image_cid = make_msgid(domain='xyz.com')
                # if `domain` argument isn't provided, it will 
                # use your computer's name

                # set an alternative html body
                msg.add_alternative(f"""
<html>
    <body>
        <p>Dear {insured_name}:<br /><br /><br />
        Regarding: Accountants Professional Liability Renewal Application<b /><br />
        <b>Policy Number:</b> {policy_number}<br /><br />
        <b>Policy Expiration Date:</b> {expiration_date}<br /><br /><br />
        Attached please find an amended Accountants Professional Liability Express renewal application for your client’s completion to renew their Accountants Professional Liability Insurance policy.   In error an outdated version of the application was previously sent to you.<br /><br />
        Please have your client use the attached renewal application to renew their Accountants Professional Liability policy, as long as they can continue to answer “true” to all the eligibility questions on the application. If for any reason they cannot continue to answer “true to all the eligibility questions on the application, please contact our office for assistance at 800-336-5422.<br /><br />
        We apologize for this error and any inconvenience this may have caused you.<br /><br />
        </p><br /><br />
        <img src="cid:{image_cid[1:-1]}">
    </body>
</html>""", subtype='html')
                # image_cid looks like <long.random.number@xyz.com>
                # to use it as the img src, we don't need `<` or `>`
                # so we use [1:-1] to strip them off


                # now open the image and attach it to the email
                with open(signature_path, 'rb') as img:

                    # know the Content-Type of the image
                    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

                    # attach it
                    msg.get_payload()[1].add_related(img.read(), 
                                                        maintype=maintype, 
                                                        subtype=subtype, 
                                                        cid=image_cid)
                
                # \D44200_1022_A_Express_App
                file_path = f'{applications_path}\D44200_1022_A_Express_App\{producer_code}_{policy_number}.pdf'
                msg.add_attachment(open(file_path, 'rb').read(), 
                                   filename=f"Renewal-Application-{policy_number_digits}.pdf", 
                                   mimetype='application/pdf')

                # send the message via our own SMTP server
                with smtplib.SMTP(smtp_host) as s:
                    s.send_message(msg)

            else:
                msg = EmailMessage()

                # generic email headers
                msg['Subject'] = 'Hello there'
                msg['From'] = 'landy_insurance@landy.com'
                msg['To'] = 'kristofer@landy.com'
                # msg['To'] = insured_email

                # set the plain text body
                msg.set_content(f'''Dear {insured_name}:
                                
Regarding:  Accountants Professional Liability Renewal Application

            Policy Number: {policy_number}

            Policy Expiration Date: {expiration_date}

 

Attached please find an amended Accountants Professional Liability Express renewal application for your completion to renew your Accountants Professional Liability Insurance policy. In error an outdated version of the application was previously sent to you.

 

Please use the attached renewal application to renew your Accountants Professional Liability policy, as long as you can continue to answer “true” to all the eligibility questions on the application. If for any reason you cannot continue to answer “true to all the eligibility questions on the application, please contact our office for assistance at 800-336-5422.

 

We apologize for this error and any inconvenience this may have caused you.''')
                
                # now create a Content-ID for the image
                image_cid = make_msgid(domain='xyz.com')
                # if `domain` argument isn't provided, it will 
                # use your computer's name

                # set an alternative html body
                msg.add_alternative(f"""
<html>
    <body>
        <p>Dear {insured_name}:<br /><br /><br />
        Regarding: Accountants Professional Liability Renewal Application<b /><br />
        <b>Policy Number:</b> {policy_number}<br /><br />
        <b>Policy Expiration Date:</b> {expiration_date}<br /><br /><br />
        Attached please find an amended Accountants Professional Liability Express renewal application for your completion to renew your Accountants Professional Liability Insurance policy. In error an outdated version of the application was previously sent to you.<br /><br />
        Please use the attached renewal application to renew your Accountants Professional Liability policy, as long as you can continue to answer “true” to all the eligibility questions on the application. If for any reason you cannot continue to answer “true to all the eligibility questions on the application, please contact our office for assistance at 800-336-5422.<br /><br />
        We apologize for this error and any inconvenience this may have caused you.<br /><br />
        </p><br /><br />
        <img src="cid:{image_cid[1:-1]}">
    </body>
</html>""", subtype='html')
                # image_cid looks like <long.random.number@xyz.com>
                # to use it as the img src, we don't need `<` or `>`
                # so we use [1:-1] to strip them off


                # now open the image and attach it to the email
                with open(signature_path, 'rb') as img:

                    # know the Content-Type of the image
                    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

                    # attach it
                    msg.get_payload()[1].add_related(img.read(), 
                                                        maintype=maintype, 
                                                        subtype=subtype, 
                                                        cid=image_cid)
                
                # \D44200_1022_A_Express_App
                file_path = f'{applications_path}\D44200_1022_D_Express_App\{producer_code}_{policy_number}.pdf'
                msg.add_attachment(open(file_path, 'rb').read(), 
                                   filename=f"Renewal-Application-{policy_number_digits}.pdf", 
                                   mimetype='application/pdf')

                # send the message via our own SMTP server
                with smtplib.SMTP(smtp_host) as s:
                    s.send_message(msg)
